In [87]:
!pip install haversine
!pip install xgboost
!pip install dython
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=7f2e7f2a9c2562f54838621f019ce0c69825dd188f0f278a2395a870c99ef7c4
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [88]:
pip uninstall sklearn

Found existing installation: sklearn 0.0
Uninstalling sklearn-0.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/sklearn-0.0.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled sklearn-0.0


In [89]:
pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [90]:
import io
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas.plotting import scatter_matrix
import xgboost as xgb
from sklearn.model_selection import train_test_split
from haversine import haversine
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,_forest
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from haversine import haversine
from dython.nominal import associations
from sklearn.utils import shuffle

In [91]:
uploaded = files.upload()

Saving data_new.csv to data_new (1).csv


In [92]:
data = pd.read_csv(io.StringIO(uploaded['data_new.csv'].decode('utf-8')),parse_dates=['datetime'])
data.head()

,latitude,longitude,speed,datetime,trackingId,company,line
0,14.696172,-17.457460,0.000000,2018-05-28 23:56:50.921000+00:00,779dbefd-0d94-450e-8971-e4ffb419e449,aftu,71
1,14.696170,-17.457467,0.000000,2018-05-28 23:59:27.354000+00:00,05d15c29-1fc6-4f29-beb1-b823ece21b70,aftu,71
2,14.696179,-17.457473,0.000000,2018-05-29 00:09:15.233000+00:00,fe38cfe5-eaab-4598-b2e9-88cb7969607a,aftu,71
3,14.693557,-17.459105,6.668741,2018-05-29 00:10:00.694000+00:00,fe38cfe5-eaab-4598-b2e9-88cb7969607a,aftu,71
4,14.692543,-17.461795,0.000000,2018-05-29 00:11:21.476000+00:00,fe38cfe5-eaab-4598-b2e9-88cb7969607a,aftu,71


In [93]:
groups = data.groupby(['trackingId'])

In [94]:
def cartesian_product_basic(left, right):
    return (left.assign(key=1).merge(right.assign(key=1), on='key').drop('key', 1))
  
temp = []
for keys, group in groups:
  #computing cartesian product of differents group and store it in temp variable
  temp.append(cartesian_product_basic(group, group))

#create a dataframe to store the dataset
dataset = pd.DataFrame(data=temp[0])
i = 1
while i<len(temp):
  df = pd.DataFrame(data = temp[i])
  dataset = pd.concat([dataset,df],axis=0,ignore_index=True)
  i = i + 1 
  
dataset.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,latitude_x,longitude_x,speed_x,datetime_x,trackingId_x,company_x,line_x,latitude_y,longitude_y,speed_y,datetime_y,trackingId_y,company_y,line_y
0,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.768950,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71
1,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.756368,-17.324461,0.0,2018-07-18 10:12:06.873000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71
2,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.725277,-17.296197,0.0,2018-07-18 10:14:45.083000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71
3,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.748355,-17.325661,0.0,2018-07-18 10:16:15.511000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71
4,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.748194,-17.325570,0.0,2018-07-18 10:18:05.636000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71


In [95]:
columns = {'latitude_x':'pickup_lat','longitude_x':'pickup_long','speed_x':'speed','datetime_x':'pickup_datetime','trackingId_x':'trackingId',
            'company_x':'company','line_x':'line','latitude_y':'dropoff_lat','longitude_y':'dropoff_long','datetime_y':'dropoff_datetime'}
dataset.rename(columns=columns, inplace=True)
dataset.head()

,pickup_lat,pickup_long,speed,pickup_datetime,trackingId,company,line,dropoff_lat,dropoff_long,speed_y,dropoff_datetime,trackingId_y,company_y,line_y
0,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.768950,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71
1,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.756368,-17.324461,0.0,2018-07-18 10:12:06.873000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71
2,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.725277,-17.296197,0.0,2018-07-18 10:14:45.083000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71
3,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.748355,-17.325661,0.0,2018-07-18 10:16:15.511000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71
4,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.748194,-17.325570,0.0,2018-07-18 10:18:05.636000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71


In [96]:
columns= ['speed_y','trackingId_y','company_y','line_y']
dataset = dataset.drop(columns,axis=1)
dataset.head()

,pickup_lat,pickup_long,speed,pickup_datetime,trackingId,company,line,dropoff_lat,dropoff_long,dropoff_datetime
0,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.768950,-17.319019,2018-07-18 10:11:05.566000+00:00
1,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.756368,-17.324461,2018-07-18 10:12:06.873000+00:00
2,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.725277,-17.296197,2018-07-18 10:14:45.083000+00:00
3,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.748355,-17.325661,2018-07-18 10:16:15.511000+00:00
4,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.748194,-17.325570,2018-07-18 10:18:05.636000+00:00


In [97]:
dataset.isnull().sum()

pickup_lat          0
pickup_long         0
speed               0
pickup_datetime     0
trackingId          0
company             0
line                0
dropoff_lat         0
dropoff_long        0
dropoff_datetime    0
dtype: int64

In [98]:
dataset['trip_duration'] = (dataset['dropoff_datetime'] - dataset['pickup_datetime']).dt.total_seconds()
#filter result by taking only positive records
dataset = dataset.query('trip_duration > 0')


In [99]:
dataset.head()

,pickup_lat,pickup_long,speed,pickup_datetime,trackingId,company,line,dropoff_lat,dropoff_long,dropoff_datetime,trip_duration
1,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.756368,-17.324461,2018-07-18 10:12:06.873000+00:00,61.307
2,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.725277,-17.296197,2018-07-18 10:14:45.083000+00:00,219.517
3,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.748355,-17.325661,2018-07-18 10:16:15.511000+00:00,309.945
4,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.748194,-17.325570,2018-07-18 10:18:05.636000+00:00,420.070
5,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.750238,-17.345286,2018-07-18 10:21:11.641000+00:00,606.075


In [100]:
def add_datepart(df, fldname, drop=True):
    fld = df[fldname]
    '''if not np.issubdtype(fld.dtypes, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld,infer_datetime_format=True)'''
    for n in ('month','day','hour','minute', 'weekday', 
              'is_month_end', 'is_month_start'):
        df[n] = getattr(fld.dt,n)
    if drop: df.drop(fldname, axis=1, inplace=True)
      
add_datepart(dataset,'pickup_datetime',drop=False)

In [101]:
dataset.head()

,pickup_lat,pickup_long,speed,pickup_datetime,trackingId,company,line,dropoff_lat,dropoff_long,dropoff_datetime,trip_duration,month,day,hour,minute,weekday,is_month_end,is_month_start
1,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.756368,-17.324461,2018-07-18 10:12:06.873000+00:00,61.307,7,18,10,11,2,False,False
2,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.725277,-17.296197,2018-07-18 10:14:45.083000+00:00,219.517,7,18,10,11,2,False,False
3,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.748355,-17.325661,2018-07-18 10:16:15.511000+00:00,309.945,7,18,10,11,2,False,False
4,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.748194,-17.325570,2018-07-18 10:18:05.636000+00:00,420.070,7,18,10,11,2,False,False
5,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.750238,-17.345286,2018-07-18 10:21:11.641000+00:00,606.075,7,18,10,11,2,False,False


In [102]:
dataset['is_weekend'] = dataset['weekday']
dataset['is_weekend'] = dataset['is_weekend'].map({0:False,1:False,2:False,3:False,4:False,5:True,6:True})
dataset.head()

,pickup_lat,pickup_long,speed,pickup_datetime,trackingId,company,line,dropoff_lat,dropoff_long,dropoff_datetime,trip_duration,month,day,hour,minute,weekday,is_month_end,is_month_start,is_weekend
1,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.756368,-17.324461,2018-07-18 10:12:06.873000+00:00,61.307,7,18,10,11,2,False,False,False
2,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.725277,-17.296197,2018-07-18 10:14:45.083000+00:00,219.517,7,18,10,11,2,False,False,False
3,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.748355,-17.325661,2018-07-18 10:16:15.511000+00:00,309.945,7,18,10,11,2,False,False,False
4,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.748194,-17.325570,2018-07-18 10:18:05.636000+00:00,420.070,7,18,10,11,2,False,False,False
5,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.750238,-17.345286,2018-07-18 10:21:11.641000+00:00,606.075,7,18,10,11,2,False,False,False


In [103]:
dataset.loc[:,'pickup_latlong'] = dataset[['pickup_lat', 'pickup_long']].apply(tuple, axis=1)
dataset.loc[:,'dropoff_latlong'] = dataset[['dropoff_lat', 'dropoff_long']].apply(tuple, axis=1)
haversine_dist = []
for index,rows in dataset.iterrows():
  temp = haversine(dataset['pickup_latlong'][index],dataset['dropoff_latlong'][index])
  haversine_dist.append(temp)
#create the column to store the result  
dataset = dataset.assign(haversine_dist=haversine_dist)

In [104]:
dataset.head()

,pickup_lat,pickup_long,speed,pickup_datetime,trackingId,company,line,dropoff_lat,dropoff_long,dropoff_datetime,...,day,hour,minute,weekday,is_month_end,is_month_start,is_weekend,pickup_latlong,dropoff_latlong,haversine_dist
1,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.756368,-17.324461,2018-07-18 10:12:06.873000+00:00,...,18,10,11,2,False,False,False,"(14.7689499, -17.3190193)","(14.7563678, -17.3244611)",1.516497
2,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.725277,-17.296197,2018-07-18 10:14:45.083000+00:00,...,18,10,11,2,False,False,False,"(14.7689499, -17.3190193)","(14.7252772, -17.2961968)",5.441088
3,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.748355,-17.325661,2018-07-18 10:16:15.511000+00:00,...,18,10,11,2,False,False,False,"(14.7689499, -17.3190193)","(14.7483549, -17.3256614)",2.398848
4,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.748194,-17.325570,2018-07-18 10:18:05.636000+00:00,...,18,10,11,2,False,False,False,"(14.7689499, -17.3190193)","(14.7481936, -17.3255697)",2.413080
5,14.76895,-17.319019,0.0,2018-07-18 10:11:05.566000+00:00,00b84e14-57f6-4319-93d0-f2fcd1f59cbf,aftu,71,14.750238,-17.345286,2018-07-18 10:21:11.641000+00:00,...,18,10,11,2,False,False,False,"(14.7689499, -17.3190193)","(14.7502378, -17.3452859)",3.508019


In [105]:
dataset = dataset.query('hour < 22 and hour > 6')
columns = ['pickup_datetime',	'trackingId',	'company','line','dropoff_datetime','pickup_latlong',	'dropoff_latlong']
dataset = dataset.drop(columns=columns,axis=1)
dataset.head()

,pickup_lat,pickup_long,speed,dropoff_lat,dropoff_long,trip_duration,month,day,hour,minute,weekday,is_month_end,is_month_start,is_weekend,haversine_dist
1,14.76895,-17.319019,0.0,14.756368,-17.324461,61.307,7,18,10,11,2,False,False,False,1.516497
2,14.76895,-17.319019,0.0,14.725277,-17.296197,219.517,7,18,10,11,2,False,False,False,5.441088
3,14.76895,-17.319019,0.0,14.748355,-17.325661,309.945,7,18,10,11,2,False,False,False,2.398848
4,14.76895,-17.319019,0.0,14.748194,-17.325570,420.070,7,18,10,11,2,False,False,False,2.413080
5,14.76895,-17.319019,0.0,14.750238,-17.345286,606.075,7,18,10,11,2,False,False,False,3.508019


In [106]:

X_train, X_test, y_train, y_test = train_test_split(dataset, dataset['trip_duration'], test_size=0.15, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15, random_state=42)
print('Training features Shape:', X_train.shape)
print('Training labels Shape:', y_train.shape)
print('Validation features Shape:', X_valid.shape)
print('Validation labels Shape:', y_valid.shape)
print('Testing featuresShape:', X_test.shape)
print('Testing labels Shape:', y_test.shape)


Training features Shape: (85128, 15)
Training labels Shape: (85128,)
Validation features Shape: (15023, 15)
Validation labels Shape: (15023,)
Testing featuresShape: (17674, 15)
Testing labels Shape: (17674,)


In [107]:
train_labels = np.array(y_train)

#features represent the predictors also named independant variables
train_features = X_train.drop(['trip_duration'],axis=1)

#save the features list 
features_list = list(train_features)

#convert to numpy array
train_features = np.array(train_features)

In [108]:
dtrain = xgb.DMatrix(train_features, label=train_labels)
dvalid = xgb.DMatrix(X_valid.drop('trip_duration',axis=1).values)
evals = [(dtrain, 'train')]
params = {
        'min_child_weight': 1, 'eta': 0.11,
        'colsample_bytree':0.2,'max_depth': 8,
        'subsample': 0.5,'lambda':0.4,
        'booster': 'gbtree', 'gamma': 0.6,
        'eval_metric':'rmse',
        'objective': 'reg:linear','n_estimators':20,
        'n_jobs':-1,'base_score':0.5
    }

#validation features
valid_features = X_valid.drop('trip_duration',axis=1)

models = []
models.append(('MultiLinearRegression', LinearRegression()))
models.append(('Random Forest', RandomForestRegressor(n_estimators=20 , min_samples_leaf=25, max_features=0.5, n_jobs=-1)))
models.append(('ExtaTreeRegressor', ExtraTreesRegressor(n_estimators=20, bootstrap=True, min_samples_leaf=25, max_features='auto', n_jobs=-1)))
models.append(('XGBoost',xgb))

results = []
names = []
for name, model in models:
 if(name=='XGBoost'):
   model= model.train(params=params, dtrain=dtrain, num_boost_round=227,evals=evals, early_stopping_rounds=300,maximize=False,verbose_eval=20)
   predictions = model.predict(dvalid)
 else:
    model.fit(train_features,train_labels)
    predictions = model.predict(valid_features)
 result = r2_score(y_valid,predictions)
 results.append(result)
 names.append(name)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but ExtraTreesRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


[19:50:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:1670.11
Will train until train-rmse hasn't improved in 300 rounds.
[20]	train-rmse:688.35
[40]	train-rmse:527.276
[60]	train-rmse:406.942
[80]	train-rmse:357.811
[100]	train-rmse:322.658
[120]	train-rmse:277.978
[140]	train-rmse:259.149
[160]	train-rmse:245.143
[180]	train-rmse:236.87
[200]	train-rmse:224.148
[220]	train-rmse:213.892
[226]	train-rmse:211.207


In [109]:
name = pd.DataFrame(data=names,columns=['Model'],index=[0,1,2,3])
score = pd.DataFrame(data=results,columns=['Score'],index=[0,1,2,3])
model_score = pd.concat([name,score],axis=1)
model_score['Score'] = model_score['Score']*100
model_score

,Model,Score
0,MultiLinearRegression,73.519723
1,Random Forest,95.979424
2,ExtaTreeRegressor,94.526878
3,XGBoost,96.227691


In [110]:
X_test.head()

,pickup_lat,pickup_long,speed,dropoff_lat,dropoff_long,trip_duration,month,day,hour,minute,weekday,is_month_end,is_month_start,is_weekend,haversine_dist
18538,14.746863,-17.377097,0.000000,14.697167,-17.465822,2579.991,5,30,7,8,2,False,False,False,11.026542
137437,14.746630,-17.377744,0.000000,14.703358,-17.461462,1977.060,10,3,7,40,2,False,False,False,10.208425
19280,14.729883,-17.440095,12.892247,14.697167,-17.465822,1745.978,5,30,7,22,2,False,False,False,4.570571
38080,14.748261,-17.353025,0.000000,14.747759,-17.376937,1745.364,8,16,11,25,3,False,False,False,2.571903
279917,14.772694,-17.317040,0.000000,14.747498,-17.326556,618.652,7,26,10,16,3,False,False,False,2.982598


In [111]:

X_test = pd.DataFrame(data=[['12.92953093',	'-77.58017063'	,'12.00000',	'12.94650176',	'-77.58003116',	'6',	'24',	'16',	'45',	'2'	,False,	True	,False	,'14.568124']],
                      columns=X_valid.drop(['trip_duration'],axis=1).columns)
dtest = xgb.DMatrix(X_test.values)
predictions = model.predict(dtest)
prediction_df = pd.DataFrame(data=predictions,columns=['trip_duration'])
prediction_df = prediction_df/60
prediction_df.head()

   
        

,trip_duration
0,11.84862


In [120]:
uploaded=files.upload()
import pandas as pd

df = pd.read_csv('testing.csv')

print(df.to_string()) 



Saving testing.csv to testing (1).csv
   pickup_lat  pickup_long  speed  dropoff_lat  dropoff_long  month  day  hour  minute  weekday  is_month_end  is_month_start  is_weekend  haversine_dist
0   12.929531   -77.580171     12    12.946502    -77.580031      6   24    16      50        2          True           False       False       14.568124
1   12.946502   -77.580031     12    12.966796    -77.574570      6   24     8      45        3          True           False       False       12.187568
2   12.966796   -77.574570     12    12.975662    -77.572864      6   27    11      12        4          True           False       False       10.478838
3   12.929531   -77.580171     12    12.966796    -77.574570      6   25    15      25        6          True           False       False        5.393898
4   12.929531   -77.580171     12    12.975662    -77.572864      6   24     4       8        2          True           False       False        2.473736
5   12.946502   -77.580031     12    1

In [121]:
import pandas as pd
df = pd.read_csv('testing.csv', delimiter=',')

list_of_csv = [list(row) for row in df.values]

print(list_of_csv)  

[[12.92953093, -77.58017063, 12, 12.94650176, -77.58003116, 6, 24, 16, 50, 2, True, False, False, 14.568124], [12.94650176, -77.58003116, 12, 12.96679624, -77.57457018, 6, 24, 8, 45, 3, True, False, False, 12.187568], [12.96679624, -77.57457018, 12, 12.97566213, -77.57286429, 6, 27, 11, 12, 4, True, False, False, 10.478838], [12.92953093, -77.58017063, 12, 12.96679624, -77.57457018, 6, 25, 15, 25, 6, True, False, False, 5.3938984], [12.92953093, -77.58017063, 12, 12.97566213, -77.57286429, 6, 24, 4, 8, 2, True, False, False, 2.4737363], [12.94650176, -77.58003116, 12, 12.97566213, -77.57286429, 6, 28, 10, 39, 6, True, False, False, 8.2837363], [12.94650176, -77.58003116, 12, 12.92953093, -77.58017063, 6, 24, 11, 43, 7, True, False, False, 4.4847667], [12.96679624, -77.57457018, 12, 12.92953093, -77.58017063, 6, 24, 16, 56, 1, True, False, False, 8.3836367], [12.96679624, -77.57457018, 12, 12.94650176, -77.58003116, 6, 24, 9, 7, 5, True, False, False, 4.4836278]]


In [122]:
import csv
lst=[]
for i in list_of_csv:
  X_test=i
  dtest = xgb.DMatrix(X_test)
  predictions = model.predict(dtest)
  prediction_df = pd.DataFrame(data=predictions,columns=['trip_duration'])
  prediction_df = prediction_df/60
  lst.append(prediction_df.values)
  print(prediction_df)
print(lst)

  
  

 



   trip_duration
0      11.133433
   trip_duration
0      12.135856
   trip_duration
0      11.940006
   trip_duration
0       5.791675
   trip_duration
0       1.983671
   trip_duration
0       9.167986
   trip_duration
0        7.89343
   trip_duration
0      17.499289
   trip_duration
0       8.077788
[array([[11.133433]], dtype=float32), array([[12.135856]], dtype=float32), array([[11.940006]], dtype=float32), array([[5.7916746]], dtype=float32), array([[1.9836706]], dtype=float32), array([[9.167986]], dtype=float32), array([[7.8934298]], dtype=float32), array([[17.499289]], dtype=float32), array([[8.077788]], dtype=float32)]


In [123]:
uploaded = files.upload()


Saving pro.csv to pro (1).csv


In [124]:
import pandas as pd

df = pd.read_csv('pro.csv')

print(df.to_string()) 



      source  pickup_lat  pickup_long destination  dropoff_lat  dropoff_long
0   jaynagar   12.929531   -77.580171      labagh    12.946502    -77.580031
1   jaynagar   12.929531   -77.580171    majestic    12.975662     12.975662
2   jaynagar   12.929531   -77.580171   chickpete    12.966796    -77.574570
3    lalbagh   12.946502   -77.580031    majestic    12.975662     12.975662
4    lalbagh   12.946502   -77.580031    jaynagar    12.929531    -77.580171
5    lalbagh   12.946502   -77.580031   chickpete    12.966796    -77.574570
6  chickpete   12.966796   -77.574570    majestic    12.975662    -77.572864
7  chickpete   12.966796   -77.574570    jaynagar    12.929531    -77.580171
8  chickpete   12.966796   -77.574570      labagh    12.946502    -77.580031


In [125]:
import pandas as pd

lis = lst
df = pd.read_csv('pro.csv')
df['trip_duration'] = lis
df.to_csv('modified_data.csv')

In [126]:
import pandas as pd

df = pd.read_csv('modified_data.csv')

print(df.to_string()) 


   Unnamed: 0     source  pickup_lat  pickup_long destination  dropoff_lat  dropoff_long  trip_duration
0           0   jaynagar   12.929531   -77.580171      labagh    12.946502    -77.580031  [[11.133433]]
1           1   jaynagar   12.929531   -77.580171    majestic    12.975662     12.975662  [[12.135856]]
2           2   jaynagar   12.929531   -77.580171   chickpete    12.966796    -77.574570  [[11.940006]]
3           3    lalbagh   12.946502   -77.580031    majestic    12.975662     12.975662  [[5.7916746]]
4           4    lalbagh   12.946502   -77.580031    jaynagar    12.929531    -77.580171  [[1.9836706]]
5           5    lalbagh   12.946502   -77.580031   chickpete    12.966796    -77.574570   [[9.167986]]
6           6  chickpete   12.966796   -77.574570    majestic    12.975662    -77.572864  [[7.8934298]]
7           7  chickpete   12.966796   -77.574570    jaynagar    12.929531    -77.580171  [[17.499289]]
8           8  chickpete   12.966796   -77.574570      labagh   

In [127]:
df.to_csv('modified_data.csv') 

In [128]:
from google.colab import files
files.download('modified_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>